In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [83]:
#計測地点のosm_idを指定
start_osm = 266396723
mid_osm = [232919754,651358177]
end_osm = 637167088
start_mesh = 50325662142
end_mesh = 50325652441

In [84]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [85]:

sttime = "2022-04-01 18:00:00"
dbname = "kddi_trkn_change2_g0"

In [86]:
datedir = "230206"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_18_param_copy  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値108.75966910187142距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値67.0距離0.0 km
50325662213
存在メッシュ：50325662213値120.44136504630703距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値108.75966910187142距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値67.0距離0.0 km
50325662232
存在メッシュ：50325662232値150.5193382021218距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値100.34798954470753距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値120.44136504630703距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値108.75966910187142距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値150.5193382021218距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値120.44136504630703距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値67.0距離0.0 km
50325662232
存在メッシュ：50325662232値150.5193382021218距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値108.75966910187142距離0.14468426940260257 km
33.7984

50325662231
存在メッシュ：50325662231値66.36310148624808距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値44.0距離0.0 km
50325662213
存在メッシュ：50325662213値72.61887308490759距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値66.36310148624808距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値44.0距離0.0 km
50325662232
存在メッシュ：50325662232値88.72620297162806距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値61.8584862042696距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値72.61887308490759距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値66.36310148624808距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値88.72620297162806距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値72.61887308490759距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値44.0距離0.0 km
50325662232
存在メッシュ：50325662232値88.72620297162806距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値66.36310148624808距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッ

50325662231
存在メッシュ：50325662231値25.302237219999373距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値18.0距離0.0 km
50325662213
存在メッシュ：50325662213値27.34493815017391距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値25.302237219999373距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値18.0距離0.0 km
50325662232
存在メッシュ：50325662232値32.60447443971528距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値23.831342434047215距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値27.34493815017391距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値25.302237219999373距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値32.60447443971528距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値27.34493815017391距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値18.0距離0.0 km
50325662232
存在メッシュ：50325662232値32.60447443971528距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値25.302237219999373距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
5032566223

50325662231
存在メッシュ：50325662231値84.15519466187267距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値57.0距離0.0 km
50325662213
存在メッシュ：50325662213値91.75148874595922距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値84.15519466187267距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値57.0距離0.0 km
50325662232
存在メッシュ：50325662232値111.31038932269121距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値78.68530467661309距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値91.75148874595922距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値84.15519466187267距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値111.31038932269121距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値91.75148874595922距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値57.0距離0.0 km
50325662232
存在メッシュ：50325662232値111.31038932269121距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値84.15519466187267距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231


50325662231
存在メッシュ：50325662231値45.691694787498825距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値32.0距離0.0 km
50325662213
存在メッシュ：50325662213値49.52175903157608距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値45.691694787498825距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値32.0距離0.0 km
50325662232
存在メッシュ：50325662232値59.38338957446615距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値42.93376706383853距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値49.52175903157608距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値45.691694787498825距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値59.38338957446615距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値49.52175903157608距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値32.0距離0.0 km
50325662232
存在メッシュ：50325662232値59.38338957446615距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値45.691694787498825距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231

...

50325662231
存在メッシュ：50325662231値57.60447443999875距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値43.0距離0.0 km
50325662213
存在メッシュ：50325662213値61.68987630034782距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値57.60447443999875距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値43.0距離0.0 km
50325662232
存在メッシュ：50325662232値72.20894887943057距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値54.66268486809443距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値61.68987630034782距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値57.60447443999875距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値72.20894887943057距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値61.68987630034782距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値43.0距離0.0 km
50325662232
存在メッシュ：50325662232値72.20894887943057距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値57.60447443999875距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メ

50325662231
存在メッシュ：50325662231値32.40974565624902距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662213
存在メッシュ：50325662213値35.60146585964673距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値32.40974565624902距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値43.81949131205513距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値30.111472553198777距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値35.60146585964673距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値32.40974565624902距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値43.81949131205513距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値35.60146585964673距離0.1851577137438622 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値43.81949131205513距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値32.40974565624902距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値2

50325662231
存在メッシュ：50325662231値111.03450818499734距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値80.0距離0.0 km
50325662213
存在メッシュ：50325662213値119.71598713823911距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値111.03450818499734距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値80.0距離0.0 km
50325662232
存在メッシュ：50325662232値142.06901636878996距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値104.78320534470068距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値119.71598713823911距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値111.03450818499734距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値142.06901636878996距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値119.71598713823911距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値80.0距離0.0 km
50325662232
存在メッシュ：50325662232値142.06901636878996距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値111.03450818499734距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
5032

50325662231
存在メッシュ：50325662231値39.86613548249898距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値28.0距離0.0 km
50325662213
存在メッシュ：50325662213値43.1855244940326距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値39.86613548249898距離0.14468426940260257 km
50325662142
存在メッシュ：50325662142値28.0距離0.0 km
50325662232
存在メッシュ：50325662232値51.732270964537335距離0.2893685387995886 km
50325662124
存在メッシュ：50325662124値37.47593145532673距離0.11554041512042086 km
50325662213
存在メッシュ：50325662213値43.1855244940326距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値39.86613548249898距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値51.732270964537335距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値43.1855244940326距離0.1851577137438622 km
50325662142
存在メッシュ：50325662142値28.0距離0.0 km
50325662232
存在メッシュ：50325662232値51.732270964537335距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値39.86613548249898距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メ

50325662231
存在メッシュ：50325662231値25.530432133124354距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662213
存在メッシュ：50325662213値27.636967467366844距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値25.530432133124354距離0.14468426940260257 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値33.06086426595638距離0.2893685387995886 km
50325662124
50325662124 is kddi none!
50325662213
存在メッシュ：50325662213値27.636967467366844距離0.1851577137438622 km
50325662231
存在メッシュ：50325662231値25.530432133124354距離0.14468426940260257 km
50325662232
存在メッシュ：50325662232値33.06086426595638距離0.2893685387995886 km
50325662213
存在メッシュ：50325662213値27.636967467366844距離0.1851577137438622 km
50325662142
50325662142 is kddi none!
50325662232
存在メッシュ：50325662232値33.06086426595638距離0.2893685387995886 km
50325662231
存在メッシュ：50325662231値25.530432133124354距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
50325662231
存在メッシュ：50325662231値27.738338957499764距離0.14468426

In [87]:
connection.close()

In [88]:
#CreateGeojsonFile